# Chapter 5. Support Vector Machines

## Notes and Code-Along

#### SVM
Well suited for complex small and medium datasets.

#### Linear SVM
* Large Margin Classification
* SVMs are sensitive to feature scales.  
**Soft Margin Classification**
* Find a good balance between keeping the street as large as possible and limiting the margin violations.
* Can be controlled in sklearn with the C hyperparameter - Smaller C - Wider Street - More margin violations - and vice versa.
* If SVM is overfitting - regularization by reducing C

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris-Virginica
svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ])
svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [2]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

**NOTE:** SVM Classifiers do not output probabilities.  
**NOTE:** Linear Kernel is slower for larger datasets. 
**NOTE:** SGDClassifier with loss hinge and alpha=1/(mC) for training a linear SVM. It does not converge faster, but helps training large datasets.  
**NOTE:** The LinearSVC class regularizes the bias term, so you should center
the training set first by subtracting its mean. Standard Scaler does this. 

#### Non-Linear SVM

**NOTE**: Non-Linear separable features - move to polinomial

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
polynomial_svm_clf = Pipeline([
("poly_features", PolynomialFeatures(degree=3)),
("scaler", StandardScaler()),
("svm_clf", LinearSVC(C=10, loss="hinge"))
])
polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('poly_features', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

**NOTE**: Low Polynomial Degree - cannot deal with complex datasets - High - model too slow.
**NOTE**: Kernel Trick - Same results as a lot of polynomials without adding them.

In [4]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='poly', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

**NOTE**: coef0 controls how much the polynomial is influenced by high-degree polynomials vs low-degree ones.

##### Adding Similarity Features
**NOTE**: Adding a feature, computed with a similarity function to measure hom much each instance resembles a particular landmark f.e. Gaussian RBF.
**NOTE**: Landmark at each instance of the dataset. 

##### Gaussian RBF Kernel
Just like polynomials - GRBF Kernels can be used to be added to the SVM.

In [5]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

**NOTE** parameter gamma controls the similarity bell curve - either narrow or wide (small gamma - wide and vice-versa)  
**NOTE** RULE OF THUMB - Try First LinearSVC and then if dataset not too large - Gaussian RBF.

**NOTE**: Linear SVC O(mxn) SVC -O(m^2xn) - slow when dealing with > 100000. Scales well with Sparse features.

#### SVM Regression
* Control the street with epsilon - high /epsilon - large street
* /epsilon-sensitive

**NOTE**: Optimization algos wwrok much better on differentiable functions.
**NOTE**: Convex Optimization with linear contraints - Quadratic Programing - Solvers Exist.  
**NOTE**: The Dual Problem - Lower bound to the solution of the primal problem. 

**NOTE**: A kernel is a function capable of computing the dot product of f(a)T f(b) based only on the original a and b.

#### **Mercer's Theorem**  
According to Mercer’s theorem, if a function K(a, b) respects a few mathematical 
conditions called Mercer’s conditions (K must be continuous, symmetric in its arguments
so K(a, b) = K(b, a), etc.), then there exists a function φ that maps a and b into
another space (possibly with much higher dimensions) such that K(a, b) = φ(a) T φ(b)

#### **Hinge Loss**
The function max(0, 1 – t) is called the hinge loss function. It is
equal to 0 when t ≥ 1. Its derivative (slope) is equal to –1 if t < 1 and 0 if t > 1. It is not differentiable at t = 1, but just like for Lasso Regression you can still use Gradient Descent using any subderivative at t = 1 (i.e., any value between –1 and 0).

## Exercises

1. What is the fundamental idea behind Support Vector Machines?
The idea is to make a decision boundary between classes within a margin of error by creating a plane h = wx + b (if x is our only feature) and having it at 0.
2. What is a support vector?
The points at which the h is at -1 or 1.
3. Why is it important to scale the inputs when using SVMs?
Because scale can impact the decision boundary.
4. Can an SVM classifier output a confidence score when it classifies an instance? What about a probability?
No for both. SVM Classifier outputs only the predicted class.
5. Should you use the primal or the dual form of the SVM problem to train a model on a training set with millions of instances and hundreds of features?
Yes - the dual form is effective when n > m.
6. Say you trained an SVM classifier with an RBF kernel. It seems to underfit the training set: should you increase or decrease γ ( gamma )? What about C ?
Increase bot C and gamma.
7. How should you set the QP parameters (H, f, A, and b) to solve the soft margin linear SVM classifier problem using an off-the-shelf QP solver?
...
8. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a SGDClassifier on the same dataset. See if you can get them to produce roughly the same model.
This could be achieved with 'hinge loss'